In [28]:
import pandas as pd

#### Ορίζω το **path** του excel αρχείου ΚΑΙ το **πλήθος** των sheets

In [29]:
# path και όνομα exel αρχείου
excel_path = 'xlsx/Greek_NRC_Emotion_Intenstity_Lex.xlsx'
# πληθος sheets
sheets_nbr = 1

#### Διαβάζω τα sheets του excel και σώζω τα dfs τους στη λίστα sheetsL

In [30]:
sheetsL = []
for i in range(0, sheets_nbr):
    sheet_df = pd.read_excel(excel_path, sheet_name=i)
    sheetsL.append(sheet_df)
    print('sheet1:', len(sheet_df), 'rows')

sheet1: 9829 rows


#### Μπορώ να καλώ τα dfs με τον παρακάτω τρόπο:

In [31]:
# 1ο sheet του excel: 1o item στη λίστα
# δηλαδή sheetsL[0] για το 1ο sheet, sheetsL[1] για το 2ο, κ.ο.κ.
sheetsL[0].head()

,English Word,Greek Word,Emotion,Emotion-Intensity-Score
0,outraged,αγανακτισμένοι,anger,0.964
1,brutality,κτηνωδία,anger,0.959
2,hatred,έχθρα,anger,0.953
3,hateful,μισητός,anger,0.940
4,terrorize,τρομοκρατώ,anger,0.939


#### Τσεκάρω για missing values

In [32]:
sheetsL[0][sheetsL[0]['English Word'].isna()]

,English Word,Greek Word,Emotion,Emotion-Intensity-Score


In [33]:
sheetsL[0][sheetsL[0]['Greek Word'].isna()]

,English Word,Greek Word,Emotion,Emotion-Intensity-Score


In [34]:
# sheetsL[0].loc[sheetsL[0]['English Word'].isna(), 'English Word'] = 'null'

#### Φέρνω και το csv με τις μεταφρασμένες λέξεις

In [35]:
translated_words_df = pd.read_csv('fixed_csvs/fixed_translated_words_2023_02.csv')
translated_words_df.head()

,English,Greek
0,abandon,εγκαταλείπω
1,abandoned,εγκαταλελειμμένο
2,abandonment,εγκατάλειψη
3,abase,παρακμή
4,abase,υποβαθμίζω


#### Ο παρακάτω κώδικας κάνει τα εξής:

- φτιάχνει ένα νέο df, όπου:
 - κάθε λέξη είναι σε ένα row και
 - τα Emotions είναι columns και τα βάρη είναι τιμές τους για την κάθε λέξη

In [36]:
# φτιάχνω λίστα με τις αγγλικές λέξεις του NRC
eil_wL = list(sheetsL[0]['English Word'].unique())
# φτιάχνω λίστα με όλα τα συναισθήματα
emoL = list(sheetsL[0]['Emotion'].unique())
# φτιάχνω κενή λίστα για να προσθέτω τα νέα sheets
transposed_sheetsL = []
# για κάθε sheet του excel (το NRC emotion intensity έχει μόνο 1)
for sheet_df in sheetsL:
    # κάνω αντίγραφο του sheet
    sheet_df_new = sheet_df.copy()
    # φτιάχνω κενή λιστα για να αποθηκεύω τα dictionaries των rows που θα κατασκευάσω
    aDsL = []
    # για κάθε αγγλική λέξη στο NRC
    for ew in eil_wL:
        # φτιάχνω ένα προσωρινό df με τα rows αυτής της λέξης
        a_df = sheet_df_new[sheet_df_new['English Word'] == ew].reset_index(drop=True)
        # παίρνω την ελληνική μετάφραση που της αντιστοιχεί
        gw = list(a_df['Greek Word'].unique())
        # φτιάχνω ένα κενό dictionary για να σώζω τις νέες τιμές (το κάθε dictionary είναι ένα row του καινούριου df που θα φτιάξω)
        aD = {}
        # η αγγλική λέξη
        aD['English'] = ew
        # η ελληνική λέξη
        aD['Greek'] = gw[0]
        # για κάθε συναίσθημα
        for emo in range(0, len(emoL)):
            # βάλε την τιμή του αν υπάρχει
            try:
                score = a_df.loc[a_df['Emotion']==emoL[emo], 'Emotion-Intensity-Score'].item()
                aD[emoL[emo]] = score
            # αν δεν υπάρχει, βάλε 0
            except:
                aD[emoL[emo]] = 0
        # πρόσθεσε το dictionary του row στη λίστα με τα dictionaries όλων των rows
        aDsL.append(aD)
    # φτιάξε ένα df από τη λίστα των dictionaries των rows
    emotion_intensity_df = pd.DataFrame(aDsL)
    # πρόσθεσε το df του corrected sheet στη λίστα
    transposed_sheetsL.append(emotion_intensity_df)
transposed_sheetsL[0]

,English,Greek,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,outraged,αγανακτισμένοι,0.964,0.0,0.000,0.000,0.0,0.000,0.0,0.000
1,brutality,κτηνωδία,0.959,0.0,0.000,0.922,0.0,0.000,0.0,0.000
2,hatred,έχθρα,0.953,0.0,0.680,0.703,0.0,0.641,0.0,0.000
3,hateful,μισητός,0.940,0.0,0.703,0.578,0.0,0.575,0.0,0.000
4,terrorize,τρομοκρατώ,0.939,0.0,0.000,0.922,0.0,0.781,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...
5886,wack,ολα,0.000,0.0,0.000,0.000,0.0,0.000,0.0,0.188
5887,insecurities,mau,0.000,0.0,0.000,0.000,0.0,0.000,0.0,0.180
5888,weaknesses,ατέλειες,0.000,0.0,0.000,0.000,0.0,0.000,0.0,0.180
5889,addict,εθίζω,0.000,0.0,0.000,0.000,0.0,0.000,0.0,0.172


#### Ο παρακάτω κώδικας κάνει τα εξής:

- μετονομάζει τις στήλες "English Word" και "Greek Word", σε "English" και "Greek" αντίστοιχα
- βλέπει τις ελληνικές μεταφράσεις από το επιμελειμένο λεξικό και προσθέτει rows με τις επιπλέον μεταφράσεις

In [37]:
# φτιάχνω λίστα με τις μεταφρασμένες αγγλικές λέξεις
eng_wordsL = list(translated_words_df['English'].unique())
# φτιάχνω κενή λίστα για να προσθέτω τα νέα sheets
new_sheetsL = []
# για κάθε sheet του excel (το NRC emotion intensity έχει μόνο 1)
for sheet_df in transposed_sheetsL:
    # φτιάχνω κενή λίστα για να προσθέτω τα dfs με τα rows που θα προσθέσω
    temp_dfsL = []
    # κάνω αντίγραφο του sheet
    sheet_df_new = sheet_df.copy()
    # διορθώνω τα ονόματα των στηλών "English" και "Greek" αν δεν είναι σωστά
    for col in sheet_df_new.columns:
        if col.startswith('English'):
            sheet_df_new.rename({col: 'English'}, axis=1, inplace=True)
        if col.startswith('Greek'):
            # κάνω πεζά τις ελληνικές λέξεις
            sheet_df_new[col] = sheet_df_new[col].str.lower()
            sheet_df_new.rename({col: 'Greek'}, axis=1, inplace=True)
    # προσθέτω το διορθωμένο sheet στη λίστα που θα βάλω και τα dfs με τα rows που θα φτιάξω μετά
    temp_dfsL.append(sheet_df_new)
    # φτιάχνω λίστα με τις αγγλικές λέξεις του VAD
    vad_wL = list(sheet_df_new['English'].unique())
    # για κάθε αγγλική λέξη από τις επιμελειμένες μεταφράσεις 
    for w in eng_wordsL:
        # αν η λέξη είναι στο VAD
        if (w in vad_wL):
            # φτιάχνω μια λίστα με τις επιμελειμένες μεταφράσεις της
            transL = translated_words_df[translated_words_df['English'] == w]['Greek'].to_list()
            # μετράω πόσες είναι οι επιμελειμένες μεταφράσεις
            ntrans = len(transL)
            # εντοπίζω τη μετάφραση της αγγλικής λέξης στο VAD
            gw = sheet_df_new.loc[sheet_df_new['English']==w]['Greek'].item()
            # για όσες μεταφράσεις τις λέξεις υπάρχουν στο επιμελειμένο λεξικό
            for i in range(0, ntrans):
                # αν η μετάφραση από το επιμελειμένο λεξικό δεν είναι ίδια με τη μετάφραση στο VAD
                if gw != transL[i]:
                    # κάνε ένα αντίγραφο του row από το VAD
                    temp_df = sheet_df_new[sheet_df_new['English'] == w].copy()
                    # και βάλε στη στήλη "Greek" την επιπλέον μετάφραση
                    temp_df['Greek'] = transL[i]
                    # και πρόσθεσε το df με αυτό το νέο row στη λίστα 
                    temp_dfsL.append(temp_df)
    # ένωσε όλα τα dfs με τα παλιά και τα νέα rows
    new_sheet_df = pd.concat(temp_dfsL, ignore_index=True)
    # κάνε string τυχόν αριθμούς
    new_sheet_df['English'] = new_sheet_df['English'].astype('str')
    # ταξινόμησε το df ως προς τις αγγλικές λέξεις
    new_sheet_df = new_sheet_df.sort_values('English').reset_index(drop=True)
    # βάλε το νεο sheet στη λίστα των νέων sheets
    new_sheetsL.append(new_sheet_df)
# συνένωσε όλα τα sheets
greek_nrc_emotion_intensity_lex_df = pd.concat(new_sheetsL, ignore_index=True)

greek_nrc_emotion_intensity_lex_df

,English,Greek,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
0,True,αληθής,0.0,0.00,0.0,0.000,0.328,0.000,0.0,0.789
1,aaaaaaah,ααααααχ,0.0,0.00,0.0,0.344,0.000,0.000,0.0,0.000
2,aaaah,ααααχ,0.0,0.00,0.0,0.234,0.000,0.000,0.0,0.000
3,abacus,άβακας,0.0,0.00,0.0,0.000,0.000,0.000,0.0,0.406
4,abandon,εγκαταλείπω,0.0,0.00,0.0,0.531,0.000,0.703,0.0,0.000
...,...,...,...,...,...,...,...,...,...,...
8869,zen,ζεν,0.0,0.00,0.0,0.000,0.515,0.000,0.0,0.000
8870,zest,πάθος,0.0,0.57,0.0,0.000,0.516,0.000,0.0,0.289
8871,zest,ζέση,0.0,0.57,0.0,0.000,0.516,0.000,0.0,0.289
8872,zest,ενθουσιασμός,0.0,0.57,0.0,0.000,0.516,0.000,0.0,0.289


In [38]:
greek_nrc_emotion_intensity_lex_df.tail(30)

,English,Greek,anger,anticipation,disgust,fear,joy,sadness,surprise,trust
8844,yayyy,ναι,0.000,0.000,0.0,0.000,0.803,0.000,0.000,0.000
8845,yayyyy,ναιιιι,0.000,0.000,0.0,0.000,0.859,0.000,0.000,0.000
8846,yeahhh,ναι,0.000,0.000,0.0,0.000,0.576,0.000,0.000,0.000
8847,yeahhhh,ναιχχχ,0.000,0.000,0.0,0.000,0.803,0.000,0.000,0.000
8848,yearning,λαχτάρα,0.000,0.711,0.0,0.000,0.297,0.000,0.000,0.398
8849,yehey,ναι,0.000,0.000,0.0,0.000,0.486,0.000,0.000,0.000
8850,yell,κραυγή,0.661,0.000,0.0,0.438,0.000,0.000,0.695,0.000
8851,yelling,δυνατή φωνή,0.788,0.000,0.0,0.000,0.000,0.000,0.000,0.000
8852,yelp,κραυγή,0.168,0.000,0.0,0.281,0.000,0.000,0.516,0.000
8853,yelp,ουρλιαχτό,0.168,0.000,0.0,0.281,0.000,0.000,0.516,0.000


#### Αποθήκευση του csv

In [39]:
# αποθήκευσε το λεξικό σαν csv
greek_nrc_emotion_intensity_lex_df.to_csv('fixed_csvs/fixed_greek_NRC_AIL_lex_2023_01.csv', index=False)